In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

In [3]:
wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16) \
.map(parse_article).flatMap(lambda x: [((x[i].lower()+'_'+x[i+1].lower()),1) for i in range(0,len(x)-1)]) \
.reduceByKey(lambda x,y:x+y) \
.filter(lambda x: x[0].startswith('narodnaya'))

result = wiki.collect()
for word in result:
    print word

(u'narodnaya_gazeta', 1)
(u'narodnaya_volya', 9)


In [5]:
#now let's make the f... grader happy
for word in result:
    print ("%s\t%d" %(word[0],word[1]) )

narodnaya_gazeta	1
narodnaya_volya	9
